In [1]:
import itk
import os
import matplotlib
import matplotlib.pyplot as plt

## Lecture des données

In [6]:
file1_path = "Data/case6_gre1.nrrd"
file2_path = "Data/case6_gre2.nrrd"

image_type = itk.Image[itk.F, 3]

reader1 = itk.ImageFileReader[image_type].New(FileName= file1_path)
reader1.Update()
image1 = reader1.GetOutput()
reader2 = itk.ImageFileReader[image_type].New(FileName= file2_path)
reader2.Update()
image2 = reader2.GetOutput()

## Recalage d'images

### Recalage par translation

In [23]:
fixed_image = image1
moving_image = image2

PixelType = itk.ctype('float')

dimension = fixed_image.GetImageDimension()
FixedImage = type(fixed_image)
MovingImage = type(moving_image)

fixed  = fixed_image
moving = moving_image

# Metric: how similarity is measured
MetricType    = itk.MeanSquaresImageToImageMetricv4[FixedImage, MovingImage]
metric        = MetricType.New()

# Optimizer: how the transform parameters are updated
OptimizerType = itk.RegularStepGradientDescentOptimizerv4[itk.D, itk.D]
opti.New()

# Registration method
RegistrationType = itk.ImageRegistrationMethodv4[FixedImage, MovingImage]
registration     = RegistrationType.New()

# Connect components
registration.SetMetric(metric)
registration.SetOptimizer(optimizer)
registration.SetFixedImage(fixed)
registration.SetMovingImage(moving)

# Choose Euler3D for rigid-body rotations + translations
TransformType = itk.Euler3DTransform[itk.D]
initialTransform = TransformType.New()

# Center the transform at the fixed image center
initialTransform.SetCenter(
    itk.CenteredTransformInitializer(
        fixed, moving, initialTransform,
        itk.CenteredTransformInitializerEnum.GEOMETRY
    ).GetTransform().GetCenter()
)

registration.SetInitialTransform(initialTransform)
registration.SetMovingInitialTransform(initialTransform)

optimizer.SetLearningRate(4.0)
optimizer.SetMinimumStepLength(0.001)
optimizer.SetNumberOfIterations(200)
optimizer.SetRelaxationFactor(0.5)

# Optional: Monitor progress
def iteration_update():
    print(f"Iteration {optimizer.GetCurrentIteration()}: "
          f"Metric = {optimizer.GetValue()}")

optimizer.AddObserver(itk.IterationEvent(), lambda evt: iteration_update())

finalTransform = registration.Execute()
print("Registration completed.")
print("Final parameters:", finalTransform.GetParameters())

ResampleFilter = itk.ResampleImageFilter[MovingImage, FixedImage]
resampler      = ResampleFilter.New(
    Input=moving,
    Transform=finalTransform,
    UseReferenceImage=True,
    ReferenceImage=fixed,
    DefaultPixelValue=0
)
resampler.Update()

aligned = resampler.GetOutput()
itk.imwrite(aligned, 'aligned_moving.nrrd')
print("Aligned image saved to aligned_moving.nrrd.")



AttributeError: 'itkImageRegistrationMethodv4REGv4F3F3' object has no attribute 'Execute'